In [ ]:
def material_properties(G, K):
  """
  Calculates Young's modulus and Poisson's ratio from shear modulus and bulk modulus.

  Args:
      G (float): The shear modulus of the material (in appropriate units).
      K (float): The bulk modulus of the material (in the same units as G).

  Returns:
      tuple: A tuple containing Young's modulus (E) and Poisson's ratio (nu).
  """

  # Calculate Young's modulus
  young_modulus = 9 * K * G / (G + 3 * K)

  # Calculate Poisson's ratio
  poisson_ratio = (3 * K - 2 * G) / (2 * G + 6 * K)

  return young_modulus, poisson_ratio


In [ ]:
from math import sqrt

def critical_toughness(E, G):
  """
  Calculates the critical toughness (Kc) from Young's modulus (E) and fracture energy rate (G).

  Args:
      E (float): Young's modulus of the material (in appropriate units).
      G (float): Fracture energy rate of the material (in compatible units with E for Kc).

  Returns:
      float: The critical toughness (Kc) of the material.
  """

  if E <= 0 or G <= 0:
    raise ValueError("Young's modulus (E) and fracture energy rate (G) must be positive values.")

  return sqrt(E * G)


In [ ]:
def clip_below_min(f, fmin):
    # Create a boolean mask where values in f are less than fmin
    mask = f < fmin

    # Use the mask to set values below fmin to fmin
    f[mask] = fmin
    
def clip_above_max(f, fmax):
    # Create a boolean mask where values in f are less than fmin
    mask = f > fmax

    # Use the mask to set values below fmin to fmin
    f[mask] = fmax

In [ ]:
# find the range of plastic failure, size of the failure range
def find_indices_2d(arr, threshold):
    """Finds the indices of the first and last elements in axis=0
    that are greater than the specified threshold for each column in axis=1.

    Args:
        arr: The 2D NumPy array.
        threshold: The value to compare against.

    Returns:
        A tuple of NumPy arrays:
            - An array of first indices for each column.
            - An array of last indices for each column.
    """
    num_rows, num_cols = arr.shape
    first_indices = np.full(num_cols, -1, dtype=int)
    last_indices = np.full(num_cols, -1, dtype=int)

    for j in range(num_cols):  # iterate over columns
        column = arr[:, j]
        greater_indices = np.where(column > threshold)[0]
        if greater_indices.size > 0:
            first_indices[j] = greater_indices[0]
            last_indices[j] = greater_indices[-1]

    return first_indices, last_indices

In [ ]:
# find the time steps when z = zp

def find_tstep(zp, dh):
    it = list(map(lambda i: i> zp, dh[:,1])).index(True)
    return it

In [ ]:
def calculate_averaged_plastic_dissipation(dataout, zmin, zmax):
    """
    Calculates the averaged plastic dissipation rate between specified depths.

    Args:
        dataout (dict): A dictionary containing the following data arrays:
            * 'dh': Displacement history (tip position over time)
            * 'wpt': Plastic dissipation rate over time
            * 'ttt': Time array
        zmin (float): The minimum depth for averaging.
        zmax (float): The maximum depth for averaging.

    Returns:
        tuple: A tuple containing:
            * wpa (float): The averaged plastic dissipation rate.
            * vt (float):  The average tip velocity over the depth range.
    """

    dh = dataout['dh']
    wp = dataout['wpt']

    # Find indices corresponding to zmin and zmax 
    it0 = np.argmin(dh[:, 1] < zmin) - 1
    it1 = np.argmin(dh[:, 1] < zmax) - 1

    # Calculate averaged plastic dissipation rate
    wpa = np.sum(wp[it0:it1]) / (it1 - it0)

    # Calculate average tip velocity
    vt = (dh[it1, 1] - dh[it0, 1]) / (dh[it1, 0] - dh[it0, 0])

    return wpa, vt

In [ ]:
def strain_rates_and_energy(data, data2, scales, params, zhead):

    pscale, vscale, tscale = scales['pscale'], scales['vscale'], scales['tscale']
    z0, z1 = zhead['z0'], zhead['z1'] # size of the head, z0 and z1: distance below and above the tip

    itmax, xc, zc, dh = data['itmax'], data['xc'], data['zc'], data['dh']

    pp, vs, us = data['pp'] * pscale, data['vzc'] * vscale, data['vxc'] * vscale
    phi, sxx, szz, sxz, dp = data['phi'], data['sxx'] * pscale, data['szz'] * pscale, data['sxz'] * pscale, data['dp'] * pscale
    exx, ezz, exz, dlam, sii = data['exx'], data['ezz'], data['exz'], data['dlam'], data['sii'] * pscale

    sxxo, szzo, sxzo, dpo = data2['sxxo'] * pscale, data2['szzo'] * pscale, data2['sxzo'] * pscale, data2['dpo'] * pscale
    vxf, vzf = data2['vxf'] * vscale, data2['vzf'] * vscale
    dd2x, dd2z, dd3x, dd3z = data2['dd2x'], data2['dd2z'], data2['dd3x'], data2['dd3z']
    dtreal = data2['dtreal'] * 3600 * 365 * 24 * 1e6 * (1e-2/tscale)

    dt = data['dt'] * 3600 * 24 * 365 * 1e3 * (1e-2/tscale)

    dx, dz = xc[1] - xc[0], zc[1] - zc[0]

    c, ct, theta, eta0, G0, Z0 = params['c'], params['ct'], params['theta'], params['eta0'], params['G0'], params['Z0']
    sint, cost, etamin, zetamax = np.sin(theta), np.cos(theta), eta0 * 1e-3, eta0 * 1e3

    # Filter phi value
    clip_below_min(phi, 1e-10)

    eta_factor = np.exp(-27 * phi)
    zeta_factor = np.copy(phi)
    clip_below_min(eta_factor, 1e-3)
    clip_below_min(zeta_factor, 1e-3)

    phis = 1 - phi
    eta, zeta = eta0 * eta_factor, eta0 / zeta_factor
    GG, ZZ = phis * G0, phis * Z0

    div = exx + ezz
    epsxx, epszz, epsxz = exx - div / 3, ezz - div / 3, np.copy(exz)

    sxxeff, szzeff, sxzeff = phis * sxx, phis * szz, phis * sxz
    dpeff = phis * dp

    epsxxv, epszzv, epsxzv = sxxeff / eta / 2, szzeff / eta / 2, sxzeff / eta / 2
    divv = -dp / zeta

    epsxxe, epszze, epsxze, dive = np.zeros_like(epsxx), np.zeros_like(epszz), np.zeros_like(epsxz), np.zeros_like(dp)


    # elastic strain rates (average in one dtck=0.01kyr)
    for it in range(np.shape(sxx)[2]):
        epsxxe[:,:,it] = (sxx[:,:,it] - sxxo[:,:,it])/2/G0/dtreal[it]
        epszze[:,:,it] = (szz[:,:,it] - szzo[:,:,it])/2/G0/dtreal[it]
        epsxze[:,:,it] = (sxz[:,:,it] - sxzo[:,:,it])/2/G0/dtreal[it]

        dive[:,:,it] = -(dp[:,:,it] - dpo[:,:,it])/Z0/dtreal[it]


    epsxxp, epszzp, epsxzp = epsxx - epsxxv - epsxxe, epszz - epszzv - epszze, epsxz - epsxzv - epsxze
    divp = div - divv - dive

    zf = 0.5 * (zc[1:] + zc[:-1])
    phifz = 0.5 * (phi[1:, :, :] + phi[:-1, :, :])
    phifx = 0.5 * (phi[:, 1:, :] + phi[:, :-1, :])

    dpdz = (pp[1:, :, :] - pp[0:-1, :, :]) / (dz * 1e3)
    dpdx = (pp[:, 1:, :] - pp[:, :-1, :]) / (dx * 1e3)

    dqz = dd2z[1:-1, :, :] * dpdz + dd3z[1:-1, :, :] * 1e3
    dvz = dqz / phifz
    vlz = vzf[1:-1, :, :] + dvz

    dqx = dd2x[:, 1:-1, :] * dpdx + dd3x[:, 1:-1, :] * 1e3
    dvx = dqx / phifx
    vlx = vxf[:, 1:-1, :] + dvx

    mphi = -dd2z[1:-1, :, :]
    wdarcy = dqz * dqz / mphi

    wvs = sxxeff * epsxxv + szzeff * epszzv + 2 * sxzeff * epsxzv
    wvd = -dpeff * divv

    wps = sxxeff * epsxxp + szzeff * epszzp + 2 * sxzeff * epsxzp
    wpd = -dpeff * divp

    wes = sxxeff * epsxxe + szzeff * epszze + 2 * sxzeff * epsxze
    wed = -dpeff * dive

    wv = np.zeros(itmax)
    we = np.zeros(itmax)
    wp = np.zeros(itmax)
    wd = np.zeros(itmax)
    tt = np.zeros(itmax)

    for it in range(itmax):
        tt[it] = dh[it,0] #tscale * it
        wv[it] = (np.sum(wvs[:, :, it]) + np.sum(wvd[:, :, it])) * (1e3 * dx) * (1e3 * dz)
        we[it] = (np.sum(wes[:, :, it]) + np.sum(wed[:, :, it])) * (1e3 * dx) * (1e3 * dz)
        wp[it] = (np.sum(wps[:, :, it]) + np.sum(wpd[:, :, it])) * (1e3 * dx) * (1e3 * dz)
        wd[it] = np.sum(wdarcy[:, :, it]) * (1e3 * dx) * (1e3 * dz)
    
    # energy in the chosen head region [zt-z0, zt+z1]
    zz0 = 0.5*(dh[1:,1]+dh[:-1,1]) - z0
    zz1 = 0.5*(dh[1:,1]+dh[:-1,1]) + z1

    iz0 = (zz0-0.5*dz)/dz
    iz1 = (zz1-0.5*dz)/dz

    iz0 = iz0.astype(int)
    iz1 = iz1.astype(int)

    # summation of total work rate
    wvt = np.zeros(itmax-1)
    wet = np.zeros(itmax-1)
    wpt = np.zeros(itmax-1)
    wdt = np.zeros(itmax-1)
    ttt = np.zeros(itmax-1)

    for it in range(itmax-1):
        ttt[it] = dh[it, 0] #0.01*it
        wvt[it] = (np.sum(wvs[iz0[it]:iz1[it],:,it]) + np.sum(wvd[iz0[it]:iz1[it],:,it])) * (1e3*dx)*(1e3*dz)
        wet[it] = (np.sum(wes[iz0[it]:iz1[it],:,it]) + np.sum(wed[iz0[it]:iz1[it],:,it])) * (1e3*dx)*(1e3*dz)
        wpt[it] = (np.sum(wps[iz0[it]:iz1[it],:,it]) + np.sum(wpd[iz0[it]:iz1[it],:,it])) * (1e3*dx)*(1e3*dz)
        wdt[it] = np.sum(wdarcy[iz0[it]:iz1[it],:,it])  * (1e3*dx)*(1e3*dz)
    
    grids = {'xc': xc, 'zc': zc}
    eps = {'exxv': epsxxv, 'exxe': epsxxe, 'exxp': epsxxp,
           'ezzv': epszzv, 'ezze': epszze, 'ezzp': epszzp,
           'exzv': epsxzv, 'exze': epsxze, 'exzp': epsxzp,
           'divv': divv, 'dive': dive, 'divp': divp}
    wlocal = {'wdarcy': wdarcy, 'wvs': wvs, 'wvd': wvd,
              'wes': wes, 'wed': wed, 'wps': wps, 'wpd': wpd}
    wglobal = {'tt': tt, 'wd': wd, 'wv': wv, 'we': we, 'wp': wp}
    whead = {'ttt': ttt, 'wdt': wdt, 'wvt': wvt, 'wet': wet, 'wpt': wpt}
    
    return grids, phi, eps, wlocal, wglobal, whead


In [ ]:
def energy_average(ta, tscale, wglobal):
    tt, wd, wv, we, wp = wglobal.values()
    
    itm = len(tt) - ta - 1

    tt2 = tscale*( ta / 2 +  (np.arange(itm) + 1))
    it_range = np.arange(itm)
    it0 = it_range + 1
    it1 = it_range + ta + 1
    wv2 = np.array([np.sum(wv[it0[i]:it1[i]]) / ta for i in range(itm)])
    we2 = np.array([np.sum(we[it0[i]:it1[i]]) / ta for i in range(itm)])
    wp2 = np.array([np.sum(wp[it0[i]:it1[i]]) / ta for i in range(itm)])
    wd2 = np.array([np.sum(wd[it0[i]:it1[i]]) / ta for i in range(itm)])
    
    return {
        'tt2': tt2,
        'wv2': wv2,
        'we2': we2,
        'wp2': wp2,
        'wd2': wd2,
    } 

In [1]:
def propagation_speed(ta, dh):
 
    itmax = np.shape(dh)[0]
    
    itm2 = itmax-ta-2
    vt = np.zeros([itm2, 2])

    for it in range(itm2):
        it0 = it
        it1 = it+ta
        vt[it,0] = 0.01*ta/2 + 0.01*(it)
        vt[it,1] = (dh[it1,1] - dh[it0,1])/(dh[it1,0] - dh[it0,0]) #(ta*1e-2)
        
    return vt

In [ ]:
def effective_toughness(young, wpt, vt):
    
    nz = np.shape(vt)[0]
    keff = np.zeros([nz,3])
    keff[:,0] = vt[:,0]
    keff[:,1] = (young*wpt/vt[:,1] *3600*24*365)**0.5/1e6
    keff[:,2] = wpt/vt[:,1] *3600*24*365/1e6
    
    return keff

In [4]:
def energy_postprocess(data, data2, scales, params, zhead, ta):

    grids, phi, eps, wlocal, wglobal, whead = strain_rates_and_energy(data, data2, scales, params, zhead)
    
    young = params['young']

    # time-averaging over the whole domain
    wg2 = energy_average(ta, scales['tscale'], wglobal)

    # time-averaging over the head region [zt-z0, zt+z1]
    wg3 = energy_average(ta, scales['tscale'], whead)
    
    newkey = ['tt3', 'wv3', 'we3', 'wp3', 'wd3']
    
    wg3=dict(zip(newkey,wg3.values()))
    
    # propagtion speed, averaging over ta steps
    vt = propagation_speed(ta, data['dh'])

    # effective toughness
    keff = effective_toughness(young, wg3['wp3'], vt)


    dataout = {**grids, **eps, **wlocal, **wglobal, **whead, **wg2, **wg3}
    dataout.update({'vt': vt, 'keff': keff, 'dh': data['dh'], 'phi': phi})

    return dataout


In [ ]:
def calculate_fracture_energy(data):
    """
    Calculates the fracture energy and related quantities for a given dataset.

    Args:
        data (dict): A dictionary containing the following keys:
            * 'itmax': Maximum number of iterations.
            * 'pp': Fluid pressure (likely a 3D array).
            * 'phi': Porosity (likely a 3D array).
            * 'dh': Height (2D array).
            * 'xc': x-coordinates (1D array).
            * 'zc': z-coordinates (1D array, in km).

    Returns:
        dict: A dictionary containing the following keys:
            * 'G': Fracture energy (2D array, shape: itmax x 2).
            * 'pl': Fluid pressure along a central vertical slice.
            * 'phi': Porosity along a central vertical slice.
            * 'iz0': Array of z-indices where fluid pressure transitions to zero. 
    """

    # Constants and conversions
    KM_TO_M = 1000  

    # Extract data and adjust units
    itmax = data['itmax']
    pp = data['pp']
    phi = data['phi']
    dh = data['dh']
    xc = data['xc'] * KM_TO_M  # Convert xc to meters
    zc = data['zc'] * KM_TO_M  # Convert zc to meters

    dz = zc[1] - zc[0]
    dx = xc[1] - xc[0]

    # Central slice index
    ii = len(xc) // 2

    # Find z-indices where fluid pressure transitions to zero 
    iz0 = np.argmax(pp[:, ii, :] > 0, axis=0)

    # Calculate porosity increment
    dphi = np.diff(phi[:, ii, :], axis=0) 

    # Calculate average fluid pressure at each z-level
    pfz = 0.5 * (pp[1:, ii, :] + pp[:-1, ii, :])

    # Calculate fracture energy (vectorized)
    dwp = np.zeros((itmax, 2))
    dwp[:, 0] = dh[:, 0]
    for i in range(itmax):
        relevant_part = pfz[iz0[i]:, i] * dphi[iz0[i]:, i]
        dwp[i, 1] = -np.sum(relevant_part) * dx 

    return {
        'G': dwp,
        'pl': pp[:, ii, :],
        'phi': phi[:, ii, :],
        'iz0': iz0
    }

In [ ]:
def calculate_average_fracture_energy(zmin, dz, iz0, wf, nt):
    """
    Calculates the average fracture energy within a specified time period.

    Args:
        zmin (float): The minimum depth of interest where pl=0.
        dz (float): The vertical discretization spacing.
        iz0 (array-like): An array containing the indices where fluid pressure transitions to zero.
        wf (array-like): An array of time-dependent fracture energy values.
        nt (int): The number of time steps to consider.

    Returns:
        float: The average fracture energy within the specified depth range and time period.
    """

    izmin = int(zmin / dz)  # Calculate the minimum index in the z-direction

    it0 = np.argmax(iz0[:] > izmin)  # Find the first time step where iz0 exceeds izmin
    it1 = it0 + nt  # Calculate the end time step

    # Average the fracture energy over the specified time period
    average_energy = np.mean(wf[it0:it1, 1]) 

    return average_energy


In [ ]:
def interpolate_data(x, y, new_x, kind='linear', fill_value='extrapolate'):
    """
    Interpolates data points onto a user-defined regular grid with extrapolation.

    Args:
        x (array-like): The original x-coordinates.
        y (array-like): The corresponding y-values.
        new_x (array-like): The new x-coordinates for interpolation.
        kind (str, optional): The kind of interpolation to use. Defaults to 'linear'.
        fill_value (str or float, optional):  Controls extrapolation behavior. 
             * 'extrapolate': Extrapolate using the chosen interpolation method.
             * A specific fill value (e.g., 0, NaN) to fill outside the original range.              

    Returns:
        ndarray: The interpolated y-values at the new x-coordinates.
    """

    interpolation_function = interp1d(x, y, kind=kind, fill_value=fill_value, bounds_error=False) 
    interpolated_y = interpolation_function(new_x)

    return interpolated_y


In [1]:
def calculate_averaged_profile(phi, pl, iz0, it0, it1, num_points=1000):
    """
    Calculates an averaged profile over a time range, interpolated onto a regular grid.

    Args:
        phi (array-like): An array of phi values.
        pl (array-like): An array of pressure values.
        iz0 (array-like): An array of indices for determining the vertical range.
        it0 (int): The starting time index.
        it1 (int): The ending time index.
        num_points (int, optional): Number of points in the output grid. Defaults to 1000.

    Returns:
        np.column_stack((x_data, y_data)) , numpy.ndarray, sample raw data before interpolation: A 2D array 
                    where the first column is the grid ('new_x') and the second column is the averaged profile ('new_y').  
        np.column_stack((new_x, new_y)) , numpy.ndarray: A 2D array where the first column is the grid ('new_x') 
                       and the second column is the averaged profile ('new_y').  
    """

    x_min = 0
    x_max = 0
    for i in range(it0, it1):
        x_max = max(x_max, max(phi[iz0[i]:, i]))

    new_x = np.linspace(x_min, x_max, num_points)
    new_y = np.zeros(num_points)
    

    for i in range(it0, it1):
        y_raw = pl[iz0[i]:, i]
        x_raw = phi[iz0[i]:, i]
        
        #define imin and imax
        imin = np.argmin(phi[:,i]>0) - iz0[i]
        imax = np.argmax(x_raw)

        # (Assuming imax and imin are defined elsewhere)
        y_data = y_raw[imax:imin+1]
        x_data = x_raw[imax:imin+1]
        
        new_y += interpolate_data(x_data, y_data, new_x, kind='linear', fill_value=0)

    new_y /= (it1 - it0)  # Calculate average

    return np.column_stack((x_data, y_data)), np.column_stack((new_x, new_y)) 

In [ ]:
def separate_and_integrate(x_data, y_data):
  """
  Separates a dataset (x, y) into two sets (y<0 and y>0) and integrates them numerically.

  Args:
      x_data (numpy.ndarray): Array containing the x-axis values.
      y_data (numpy.ndarray): Array containing the y-axis values.

  Returns:
      tuple: A tuple containing two elements:
          - y_data_pos (numpy.ndarray): The y-axis values for y>=0.
          - y_data_neg (numpy.ndarray): The y-axis values for y<0.
          - integral_pos (float): The numerical integral of y_data_pos.
          - integral_neg (float): The numerical integral of y_data_neg. 
  """

  y_data_pos = y_data[y_data >= 0]
  x_data_pos = x_data[y_data >= 0]

  y_data_neg = y_data[y_data < 0]
  x_data_neg = x_data[y_data < 0]

  # Integrate using trapezoidal rule
  integral_pos = trapz(y_data_pos, x_data_pos)
  integral_neg = trapz(y_data_neg, x_data_neg)

  return integral_pos, integral_neg



In [ ]:
def process_data_files(filenames, keys, datapath, param, suffix='.npz'):
    """
    Processes multiple data files, calculates fracture energies and related quantities, 
    and stores results in a dictionary using provided keys.

    Args:
        filenames (list): List of base filenames (without the suffix).
        keys (list): List of corresponding keys for the results dictionary.
        datapath (str): Path to the directory containing the data files.
        param (dict): Dictionary containing processing parameters:
            * 'zmin' (float): Minimum depth for averaging fracture energy.
            * 'nt' (int): Number of time steps for averaging.
            * 'ngrids' (int): Number of grids for pl-phi curve averaging.
        suffix (str, optional): The filename suffix. Defaults to '.npz'.

    Returns:
        dict: A dictionary where provided keys map to dictionaries containing calculated data: 
    """
    
    zmin = param['zmin']
    nt = param['nt']
    ngrids = param['ngrids']

    results_dict = {}
    for filename, key in zip(filenames, keys):
        full_filename = datapath + filename + suffix
        data = np.load(full_filename, allow_pickle=True)
        dataout = calculate_fracture_energy(data) # contains timedependent values: iz0[nt], G[nt,2], phi[nz,nt], pl[nz,nt] 

        # prepare variables
        dz = data['zc'][1] - data['zc'][0]
        print('case:', filename, 'dz=', dz)
        
        iz0 = dataout['iz0']
        wf = dataout['G']
        pl = dataout['pl']
        phi = dataout['phi']
        dh = data['dh']

        # extract izt[nt]
        izt = dh[:,1]/dz + 1
        dataout['izt'] = izt
        
        # time averaged G
        wfa = calculate_average_fracture_energy(zmin, dz, iz0, wf, nt) 
        dataout['Ga'] = wfa
        
        # taking the averaged curve of pl-phi by: define a fixed grids of phi, interpolate pl over the grids, 
        #      and taking time-averaged value of pl on each grid point of phi
        ## find it0 and it1 to take average##
        izmin = int(zmin / dz)  # Calculate the minimum index in the z-direction
        it0 = np.argmax(iz0[:] > izmin)  # Find the first time step where iz0 exceeds izmin
        it1 = it0 + nt  # Calculate the end time step
        xy_end, pphi_new = calculate_averaged_profile(phi, pl, iz0, it0, it1, num_points=ngrids)

        dataout['pl_phi_raw'] = xy_end
        dataout['pl_phi_av'] = pphi_new
        
        # the averaged distance between the tip and the location firstly appear pl>0 upwards (zt - zc[iz0])
        zl = dz * np.sum(izt[it0:it1] - iz0[it0:it1])/nt
        
        # averaged size of plastic failure head
        ii = int(len(data['xc'])/2)
        izp0, izp1 = find_indices_2d(data['dlam'][:,ii,:], 1e-12)

        zpl = dz * np.sum(izp1[it0:it1]-izp0[it0:it1])/nt
                
        dataout['head'] =[zl, zpl]
        
        # loading and unloading energy according to the averaged curve of pl-phi
        wf_pos, wf_neg = separate_and_integrate(pphi_new[:,0], pphi_new[:,1])
        
        ip0 = np.argmax(pphi_new[:,1]>=0)
        print('ip0 = ',ip0)
        phi_trans = pphi_new[ip0,0]
        
        dataout['load_unload'] = [wf_pos, wf_neg, phi_trans]
        
        # maximum phi and pl according to the averaged curve of pl-phi
        dataout['maxval'] = [np.max(pphi_new[:,0]), np.max(pphi_new[:,1])]
        
        # the averaged speed in the range t=[it0:it1]
        vt = (dh[it1,1] - dh[it0,1])/(dh[it1,0] - dh[it0,0])
        dataout['vt'] =vt

        
        results_dict[key] = dataout

    return results_dict